In [52]:
import numpy as np

In [ ]:
# Problem 1: Binary Words and Operations

def Parity(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return x ^ y ^ z

def Ch(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (~x & z)

def Maj(x, y, z):
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    return (x & y) ^ (x & z) ^ (y & z)

def Sigma0(x):
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_2 = (x >> 2) | (x << (32 - 2))
    rotr_13 = (x >> 13) | (x << (32 - 13))
    rotr_22 = (x >> 22) | (x << (32 - 22))
    return rotr_2 ^ rotr_13 ^ rotr_22

def Sigma1(x):
    x = np.uint32(x)
    # Inline rotation implementation
    rotr_6 = (x >> 6) | (x << (32 - 6))
    rotr_11 = (x >> 11) | (x << (32 - 11))
    rotr_25 = (x >> 25) | (x << (32 - 25))
    return rotr_6 ^ rotr_11 ^ rotr_25

def sigma0(x):
    x = np.uint32(x)
    # Inline implementation
    rotr_7 = (x >> 7) | (x << (32 - 7))
    rotr_18 = (x >> 18) | (x << (32 - 18))
    shr_3 = x >> 3
    return rotr_7 ^ rotr_18 ^ shr_3

def sigma1(x):
    x = np.uint32(x)
    # Inline implementation
    rotr_17 = (x >> 17) | (x << (32 - 17))
    rotr_19 = (x >> 19) | (x << (32 - 19))
    shr_10 = x >> 10
    return rotr_17 ^ rotr_19 ^ shr_10

"Testing 7 required functions**************************************************"

# Test the 7 required functions
print(f"Parity(0x00000001, 0x00000001, 0x00000000) = {Parity(0x00000001, 0x00000001, 0x00000000):08x}")
print(f"Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = {Ch(0xFFFFFFFF, 0x12345678, 0x87654321):08x}")
print(f"Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = {Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F):08x}")
print()

test_val = 0x12345678
print(f"Sigma0(0x12345678) = {Sigma0(test_val):08x}")
print(f"Sigma1(0x12345678) = {Sigma1(test_val):08x}")
print(f"sigma0(0x12345678) = {sigma0(test_val):08x}")
print(f"sigma1(0x12345678) = {sigma1(test_val):08x}")

Parity(0x00000001, 0x00000001, 0x00000000) = 00000000
Ch(0xFFFFFFFF, 0x12345678, 0x87654321) = 12345678
Maj(0xF0F0F0F0, 0xFF00FF00, 0x0F0F0F0F) = ff00ff00

Sigma0(0x12345678) = 66146474
Sigma1(0x12345678) = 3561abda
sigma0(0x12345678) = e7fce6ee
sigma1(0x12345678) = a1f78649
